In [1]:
!git clone https://github.com/nkt780426/concat.git

Cloning into 'concat'...
remote: Enumerating objects: 575, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 575 (delta 36), reused 111 (delta 23), pack-reused 445 (from 1)
Receiving objects: 100% (575/575), 79.56 MiB | 46.13 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [2]:
!gdown --quiet --folder "https://drive.google.com/drive/folders/111XLjyrsaLIAu4BsW8ZhLa68vnWLDLnH?usp=sharing"

In [3]:
!mv concat/going_modular .

In [3]:
import warnings
warnings.filterwarnings("ignore")
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A


from going_modular.model.TripletFaceRecognition import EmbeddingNet_Concat_V2, TripletNet_Concat_V2
from going_modular.dataloader.triplet import TripletDatasetConcatV2
from going_modular.loss.TripletLoss import TripletLoss
from going_modular.dataloader.triplet import CustomExrDatasetConCatV2
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise
from going_modular.train_eval.triplet.train import fit

import os

import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    'type': 'concat',
    # Thư mục
    'data_dir': './Dataset',
    'checkpoint_dir': './checkpoint/test',

    # Cấu hình train
    'epochs': 348,
    'num_workers': 4,
    'batch_size': 4,
    'image_size': 256,
    'embedding_size': 512,
    
    'start_lr': 1e-4,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # triplet
    'margin': 1.,
}


train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
], additional_targets={
    'albedo': 'image',
    'depthmap': 'image'
})


test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
], additional_targets={
    'albedo': 'image',
    'depthmap': 'image'
})

triplet_concat_train_dataset = TripletDatasetConcatV2(data_dir=CONFIGURATION['data_dir'], transform=train_transform, train=True)
triplet_concat_test_dataset = TripletDatasetConcatV2(data_dir=CONFIGURATION['data_dir'], transform=test_transform, train=False)


triplet_concat_train_loader = DataLoader(
    triplet_concat_train_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)


triplet_concat_test_loader = DataLoader(
    triplet_concat_test_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    drop_last=True
)


roc_concat_train_dataset = CustomExrDatasetConCatV2(data_dir=CONFIGURATION['data_dir'], transform=train_transform, train=True)
roc_concat_train_loader = DataLoader(
    roc_concat_train_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

roc_concat_test_dataset = CustomExrDatasetConCatV2(data_dir=CONFIGURATION['data_dir'], transform=test_transform, train=False)
roc_test_loader = DataLoader(
    roc_concat_test_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

embedding_net = EmbeddingNet_Concat_V2(conf=CONFIGURATION)

model = TripletNet_Concat_V2(embedding_net).to(device)
    
criterion = TripletLoss(CONFIGURATION['margin'])
optimizer = Adam(model.parameters(), lr=CONFIGURATION['start_lr'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=30, T_mult=1, eta_min=1e-6)

earlystop_dir = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models')

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'euclidean_auc'],
    patience=1000,
    mode='max',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=0
)      

checkpoint_path = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)

In [4]:
checkpoint = torch.load('./checkpoint/new/' + CONFIGURATION['type'] + '/models/checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

In [5]:
# Kiểm tra trạng thái của resnet1
for name, param in model.embedding_net.resnet1.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

# Kiểm tra trạng thái của resnet2
for name, param in model.embedding_net.resnet2.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

# Kiểm tra trạng thái của resnet3
for name, param in model.embedding_net.resnet3.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

conv1.weight: requires_grad=True
bn1.weight: requires_grad=True
bn1.bias: requires_grad=True
layer1.0.conv1.weight: requires_grad=True
layer1.0.bn1.weight: requires_grad=True
layer1.0.bn1.bias: requires_grad=True
layer1.0.conv2.weight: requires_grad=True
layer1.0.bn2.weight: requires_grad=True
layer1.0.bn2.bias: requires_grad=True
layer1.1.conv1.weight: requires_grad=True
layer1.1.bn1.weight: requires_grad=True
layer1.1.bn1.bias: requires_grad=True
layer1.1.conv2.weight: requires_grad=True
layer1.1.bn2.weight: requires_grad=True
layer1.1.bn2.bias: requires_grad=True
layer2.0.conv1.weight: requires_grad=True
layer2.0.bn1.weight: requires_grad=True
layer2.0.bn1.bias: requires_grad=True
layer2.0.conv2.weight: requires_grad=True
layer2.0.bn2.weight: requires_grad=True
layer2.0.bn2.bias: requires_grad=True
layer2.0.downsample.1.weight: requires_grad=True
layer2.0.downsample.2.weight: requires_grad=True
layer2.0.downsample.2.bias: requires_grad=True
layer2.1.conv1.weight: requires_grad=True


In [6]:
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    triplet_train_loader=triplet_concat_train_loader, 
    triplet_test_loader=triplet_concat_test_loader, 
    criterion=criterion,
    optimizer=optimizer, 
    scheduler=scheduler, 
    epochs=CONFIGURATION['epochs'], 
    device=device,
    roc_train_loader=roc_concat_train_loader, 
    roc_test_loader=roc_test_loader,
    early_stopping=early_stopping,
    model_checkpoint=modle_checkpoint
)

Epoch 1:
	train: loss: 75.8363 | auc_cos: 0.8581 | auc_eu: 0.8645
	test: loss: 124.6459 | auc_cos: 0.8406 | auc_eu: 0.8563
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/concat/checkpoint/testconcat/models/checkpoint.pth
Epoch 2:
	train: loss: 50.4921 | auc_cos: 0.9162 | auc_eu: 0.9238
	test: loss: 35.9985 | auc_cos: 0.9137 | auc_eu: 0.9198
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/concat/checkpoint/testconcat/models/checkpoint.pth
Epoch 3:
	train: loss: 32.9940 | auc_cos: 0.9531 | auc_eu: 0.9481
	test: loss: 31.3107 | auc_cos: 0.9298 | auc_eu: 0.9091
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/concat/checkpoint/testconcat/models/checkpoint.pth
Epoch 4:
	train: loss: 11.3083 | auc_cos: 0.9603 | auc_eu: 0.9648
	test: loss: 18.4063 | auc_cos: 0.9396 | auc_eu: 0.9380
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/co

KeyboardInterrupt: 

In [7]:
!zip -r output.zip checkpoint/new/concat

  adding: checkpoint/new/concat/ (stored 0%)
  adding: checkpoint/new/concat/models/ (stored 0%)
  adding: checkpoint/new/concat/models/best_cosine_auc_301.pth (deflated 7%)
  adding: checkpoint/new/concat/models/best_euclidean_auc_122.pth (deflated 7%)
  adding: checkpoint/new/concat/models/checkpoint.pth (deflated 7%)
  adding: checkpoint/new/concat/logs/ (stored 0%)
  adding: checkpoint/new/concat/logs/Cosine_auc_test/ (stored 0%)
  adding: checkpoint/new/concat/logs/Cosine_auc_test/events.out.tfevents.1734900617.28cee1f0910b.21.4 (deflated 64%)
  adding: checkpoint/new/concat/logs/Loss_test/ (stored 0%)
  adding: checkpoint/new/concat/logs/Loss_test/events.out.tfevents.1734900617.28cee1f0910b.21.2 (deflated 61%)
  adding: checkpoint/new/concat/logs/Euclidean_auc_train/ (stored 0%)
  adding: checkpoint/new/concat/logs/Euclidean_auc_train/events.out.tfevents.1734900617.28cee1f0910b.21.5 (deflated 66%)
  adding: checkpoint/new/concat/logs/Cosine_auc_train/ (stored 0%)
  adding: checkp